# 通过PAI Python SDK使用Swift微调和部署大语言模型


[ModelScope Swift](https://github.com/modelscope/swift) 是 ModelScope 社区开发的高效微调训练和推理框架，他支持一系列的先进的开源大语言模型的微调训练和部署，包括 `Qwen`，`Mixtral`，`Baichuan`，`ChatGLM`，`LLama2`等，开发者可以通过数行代码即可完成大语言模型的微调和部署。

在当前文档中，我们将以[qwen/Qwen1.5-7B-Chat](https://modelscope.cn/models/qwen/Qwen1.5-7B-Chat/summary) 模型为示例，展示如何通过[PAI Python SDK](https://alipai.readthedocs.io/)在PAI平台上使用ModelScope Swift进行微调训练和模型部署。


## 前提准备


安装PAI Python SDK，用于提交任务或是部署推理服务。

In [ ]:
# 安装PAI Python SDK
!python -m pip install -U "alipai"



SDK需要配置访问阿里云服务需要的AccessKey，以及当前使用的工作空间和OSS Bucket。在PAI SDK安装之后，通过在**命令行终端** 中执行以下命令，按照引导配置密钥、工作空间等信息。


```shell

# 以下命令，请在 "命令行终端" 中执行.

python -m pai.toolkit.config

```

我们可以执行以下代码，验证配置是否成功。

In [ ]:
import pai
from pai.session import get_default_session, setup_default_session

print(pai.__version__)

sess = get_default_session()

# 用户也可以通过代码方式配置AK/SK/Region/WorkspaceId等信息
# if not sess:
#     sess = setup_default_session(
#         access_key_id="<your-access-key-id>",
#         access_key_secret="<your-access-key-secret>",
#         region_id="<region-id>",
#         workspace_id="<workspace-id>",
#         oss_bucket_name="<oss-bucket-name>",
#     )
#     sess.save_config()


# 配置成功之后，我们可以拿到工作空间的信息
assert sess is not None
assert sess.workspace_name is not None
assert sess.oss_bucket is not None


## 模型微调训练

Swifit对常见的大语言模型提供了开箱即用的参数配置和脚本，支持LoRA，全参数等方式对模型进行微调训练。
例如以下Swift提供的微调训练脚本，将使用`ms-bench-mini`数据集，对`Qwen1.5-7B-Chat`模型进行LoRA微调训练。

```shell
# source: https://github.com/modelscope/swift/blob/main/examples/pytorch/llm/scripts/qwen1half_7b_chat/lora/sft.sh

# Experimental environment: A100
# 30GB GPU memory
PYTHONPATH=../../.. \
CUDA_VISIBLE_DEVICES=0 \
python llm_sft.py \
    --model_id_or_path qwen/Qwen1.5-7B-Chat
    --model_revision master,
    --sft_type lora \
    --tuner_backend swift \
    --dtype AUTO \
    --output_dir output \
    --dataset ms-bench-mini \
    --train_dataset_sample 5000 \
    --num_train_epochs 2 \
    --max_length 1024 \
    --check_dataset_strategy warning \
    --lora_rank 8 \
    --lora_alpha 32 \
    --lora_dropout_p 0.05 \
    --lora_target_modules ALL \
    --gradient_checkpointing true \
    --batch_size 1 \
    --weight_decay 0.01 \
    --learning_rate 1e-4 \
    --gradient_accumulation_steps 16 \
    --max_grad_norm 0.5 \
    --warmup_ratio 0.03 \
    --eval_steps 100 \
    --save_steps 100 \
    --save_total_limit 2 \
    --logging_steps 10 \
    --use_flash_attn false \
    --self_cognition_sample 1000 \
    --model_name 卡卡罗特 \
    --model_author 陶白白 \
    --push_to_hub false \
    --hub_model_id qwen1half-7b-chat-lora \
    --hub_private_repo true \
    --hub_token 'your-sdk-token' \

```

以上命令中的参数详解，用户可以参考Swift文档：[支持的模型和数据集文档](https://github.com/modelscope/swift/blob/main/docs/source/LLM/%E6%94%AF%E6%8C%81%E7%9A%84%E6%A8%A1%E5%9E%8B%E5%92%8C%E6%95%B0%E6%8D%AE%E9%9B%86.md)，[微调参数详解文档](https://github.com/modelscope/swift/blob/main/docs/source/LLM/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.md)。

### 使用PAI Python SDK提交微调训练任务

通过PAI Python SDK提供的`ModelScopeEstimator`对象，用户可以方便得使用PAI提供的镜像提交训练作业。预置镜像中安装了基础的依赖库，包括`ModelScope`, `Swift`, `PyTorch`等，使用`ModelScopeEstimator`，用户可以轻松得在PAI上使用Swift框架完成模型微调训练。


以下代码中，我们基于Swift提供的`Qwen1.5-7B-Chat`模型的[LoRA训练脚本](https://github.com/modelscope/swift/blob/main/examples/pytorch/llm/scripts/qwen_7b_chat/lora/sft.sh)提供的训练参数，通过PAI Python SDK提交训练任务。

在代码中，我们通过`hyerparameters`参数，将训练参数传递给到训练作业，然后在启动命令中，使用环境变量的方式引用这些参数。

In [6]:
# 通过 git_config 指定训练脚本相关的 git 地址和分支
git_config = {"repo": "https://github.com/modelscope/swift.git", "branch": "v1.5.3"}

# Swift的LLM SFT的完整参数支持，请参考：
# https://github.com/modelscope/swift/blob/main/docs/source/LLM/%E5%91%BD%E4%BB%A4%E8%A1%8C%E5%8F%82%E6%95%B0.md#sft-%E5%8F%82%E6%95%B0
hyperparameters = {
    "model_id_or_path": "qwen/Qwen1.5-7B-Chat",
    "model_revision": "master",
    "sft_type": "lora",
    "tuner_backend": "swift",
    "dtype": "AUTO",
    "dataset": "blossom-math-zh",
    # "dataset": "ms-bench-mini",
    "train_dataset_sample": -1,
    "num_train_epochs": 1,
    "max_length": 1024,
    "check_dataset_strategy": "warning",
    "lora_rank": 8,
    "lora_alpha": 32,
    "lora_dropout_p": "0.05",
    "lora_target_modules": "DEFAULT",
    "gradient_checkpointing": "true",
    "batch_size": "4",
    "weight_decay": "0.01",
    "learning_rate": "1e-4",
    "gradient_accumulation_steps": "16",
    "max_grad_norm": "0.5",
    "warmup_ratio": "0.03",
    "eval_steps": "100",
    "save_steps": "100",
    "save_total_limit": "2",
    "logging_steps": "10",
    "use_flash_attn": "false",
}

# 训练作业需要需要配置训练输出路径
hyperparameters.update(
    {
        # 模型输出地址，请勿修改
        "output_dir": "/ml/output/model/",
    }
)

In [ ]:
from pai.modelscope import ModelScopeEstimator


# 创建 ModelScopeEstimator 对象
est = ModelScopeEstimator(
    # 指定训练脚本的启动命令
    # 1. Qwen1.5-7B-Chat 需要 ms-swift>=1.6.1 和 transformers>=4.37
    # 2. 通过 $PAI_USER_ARGS 环境变量传入所有超参信息
    command="python -m pip install --upgrade ms-swift 'transformers>=4.37'  && swift sft $PAI_USER_ARGS",
    # instance_type="ecs.gn6e-c12g1.3xlarge",  # 1xV100 GPU (32GB显存)
    instance_type="ecs.gn7e-c16g1.4xlarge",
    # 用于选择训练镜像
    modelscope_version="1.12.0",
    hyperparameters=hyperparameters,
    base_job_name="modelscope-swift-train",
)

# 提交训练作业
est.fit(wait=False)

# 打开TensorBoard作业
tb = est.tensorboard()

# 打印TensorBoard应用页面
print(tb.app_uri)

In [11]:
# 等待到训练作业结束
est.wait()

# 查看训练任务所产出的模型地址，用户可以通过ossutils，或是其他方式下载模型到本地.
print(est.model_data())

TrainingJob launch starting
LIBRARY_PATH=/usr/local/cuda/lib64/stubs
DSW_95221_PORT_22_TCP_PROTO=tcp
DSW_95221_SERVICE_PORT_SSH_DSW_95221=22
DSW_95221_PORT_80_TCP_ADDR=10.192.9.246
NV_CUDA_COMPAT_PACKAGE=cuda-compat-12-1
NV_LIBCUBLAS_VERSION=12.1.0.26-1
NV_NVPROF_DEV_PACKAGE=cuda-nvprof-12-1=12.1.55-1
KUBERNETES_PORT=tcp://10.192.0.1:443
KUBERNETES_SERVICE_PORT=6443
NV_CUDA_NSIGHT_COMPUTE_VERSION=12.1.0-1
LANGUAGE=zh_CN.UTF-8
PIP_TRUSTED_HOST=mirrors.cloud.aliyuncs.com
SCRAPE_PROMETHEUS_METRICS=yes
MASTER_ADDR=trainpiyqdicammv-master-0
DSW_95221_PORT_80_TCP_PORT=80
PAI_HPS_SFT_TYPE=lora
HOSTNAME=trainpiyqdicammv-master-0
DSW_98084_SERVICE_PORT=80
DSW_95221_SERVICE_PORT_HTTP_DSW_95221=80
DSW_95221_PORT_80_TCP_PROTO=tcp
DSW_98084_PORT=tcp://10.192.30.20:80
LD_LIBRARY_PATH=/usr/local/nvidia/lib:/usr/local/nvidia/lib64
NV_LIBNCCL_PACKAGE_VERSION=2.17.1-1
NVIDIA_CUDA_END_OF_LIFE=1
DSW_107274_SERVICE_PORT=80
DSW_107274_PORT=tcp://10.192.12.78:80
MASTER_PORT=23456
DSW_96358_PORT=tcp://10.192.

以上训练作业中，我们通过`--dataset`参数，使用了预置的数据集`blossom-math-zh`，Swift框架将自动完成数据集的下载准备工作。

Swift也支持使用自定义数据集完成微调训练，通过`--custom_train_dataset_path`和`--custom_val_dataset_path`参数，用户可以指定自定义数据集的路径，具体介绍可以参考Swift的文档：[自定义数据集微调训练](https://github.com/modelscope/swift/blob/main/docs/source/LLM/%E8%87%AA%E5%AE%9A%E4%B9%89%E4%B8%8E%E6%8B%93%E5%B1%95.md#%E8%87%AA%E5%AE%9A%E4%B9%89%E6%95%B0%E6%8D%AE%E9%9B%86)

当开发者使用`ModelScopeEstimator`在PAI提交训练作业时，支持在训练作业中使用OSS、NAS、MaxCompute表等数据源。通过`fit`方法的`inputs`参数，我们可以指定训练使用的数据集的路径，相应的数据会被准备到训练作业环境中，支持训练作业直接读取使用，具体可以参考文档：[使用训练数据](https://alipai.readthedocs.io/zh/latest/user-guide/training/use-data.html)


```python

from pai.modelscope import ModelScopeEstimator


hps = {
	"custom_train_dataset_path": "/ml/input/data/train/<TRAIN_FILE_NAME>",
	"custom_val_dataset_path": "/ml/input/data/validation/<VALIDATION_FILE_NAME>",
	# more parameters
	# ...
}
est = ModelScopeEstimator(
	# 通过 --custom_train_dataset_path 和 --custom_val_dataset_path 参数，传递数据集路径
	command="python llm_sft.py $PAI_USER_ARGS",
	# more parameters...
)


# 使用自定义数据集微调训练
est.fit(
	inputs={
		# 可以使用本地路径或者OSS路径，相应的数据集会被准备到 /ml/input/data/{channel_name}/ 路径下.
		"train": "oss://<YourOssBucketName>/<PathToTrainData>",
		# 本地文件会被上传到OSS Bucket，然后再被挂载到训练作业中。
		"validation": "/path/to/validation/data",
	}
)
```



In [ ]:
# 训练任务完成之后，删除TensorBoard实例，释放资源（每一个账号下最多同时能够开通5个免费的TensorBoard实例）
tb.delete()

## 模型部署

Swift提供了命令行工具，支持开发者将模型部署为在线推理服务，具体介绍可以参考文档：[Swift：vLLM推理加速与部署](https://github.com/modelscope/swift/blob/main/docs/source/LLM/VLLM%E6%8E%A8%E7%90%86%E5%8A%A0%E9%80%9F%E4%B8%8E%E9%83%A8%E7%BD%B2.md#%E9%83%A8%E7%BD%B2)。在本章节中，我们将通过 PAI Python SDK 将训练产出的模型部署到 PAI-EAS，创建在线推理服务。


### 下载合并模型

在模型部署之前，我们需要将微调训练获得的LoRA模型下载到本地，与原始模型合并获得完整的模型。

In [1]:
from pai.common.oss_utils import download
import glob
import os


# 下载模型到本地
local_model_dir = download(est.model_data(), "./qwen-lora-model")


# 获取模型的最新的一个checkpoint用于部署
checkpoint_dirs = glob.glob(os.path.join(local_model_dir, "checkpoint-*"))
latest_version = max(int(os.path.basename(d).split("-")[1]) for d in checkpoint_dirs)
latest_checkpoint_dir = os.path.join(local_model_dir, f"checkpoint-{latest_version}")

print(latest_checkpoint_dir)

/opt/conda/lib/python3.10/site-packages/pai/common/oss_utils.py:30: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm
Downloading: pai/training_job/modelscope_sdk_train_20240205_233710_7p4x8h/model/: 100%|██████████| 48/48 [00:06<00:00,  7.21it/s]

./qwen-lora-model/checkpoint-618



以下代码中，我们将使用Swift提供的命令行工具，将训练获得的LoRA模型和原始的模型合并，获得完整的模型。


In [ ]:
# 安装ModelScope Swift
!python -m pip install -q -U ms-swift

# 使用Swift merge-lora 工具合并模型
!swift merge-lora --ckpt_dir {latest_checkpoint_dir}


merged_model_dir = os.path.join(local_model_dir, f"checkpoint-{latest_version}-merged")
print(merged_model_dir)

# 查看合并后的模型
!ls {merged_model_dir}

上传模型到OSS Bucket，供后续推理服务加载使用。

In [4]:
from pai.common.oss_utils import upload


# 上传模型到当前session的bucket.
model_data_uri = upload(
    merged_model_dir,
    "modelscope-swift-example/qwen-lora-merged-model/",
)
print(model_data_uri)

Uploading file: qwen-lora-model/checkpoint-618-merged/configuration_qwen.py: 100%|██████████| 2.35k/2.35k [00:00<00:00, 30.6kB/s]
Uploading file: qwen-lora-model/checkpoint-618-merged/model.safetensors.index.json: 100%|██████████| 19.5k/19.5k [00:00<00:00, 864kB/s]
Uploading file: qwen-lora-model/checkpoint-618-merged/generation_config.json: 100%|██████████| 275/275 [00:00<00:00, 14.7kB/s]
Uploading file: qwen-lora-model/checkpoint-618-merged/model-00001-of-00004.safetensors: 100%|██████████| 4.99G/4.99G [00:30<00:00, 161MB/s] 
Uploading file: qwen-lora-model/checkpoint-618-merged/modeling_qwen.py: 100%|██████████| 55.6k/55.6k [00:00<00:00, 2.28MB/s]
Uploading file: qwen-lora-model/checkpoint-618-merged/configuration.json: 100%|██████████| 76.0/76.0 [00:00<00:00, 3.51kB/s]
Uploading file: qwen-lora-model/checkpoint-618-merged/qwen.tiktoken: 100%|██████████| 2.56M/2.56M [00:00<00:00, 30.8MB/s]
Uploading file: qwen-lora-model/checkpoint-618-merged/sft_args.json: 100%|██████████| 2.72k/2.

### 部署模型

通过SDK提供的`ModelScopeEstimator`对象，用户可以配置部署的模型，镜像，机器实例规格等参数，将模型部署为在线推理服务。

以下代码中，我们将使用PAI预置的`vLLM`推理服务镜像，使用Swift部署合并后的模型为一个在线推理服务。



In [7]:
from pai.modelscope import ModelScopeModel
from pai.common.utils import random_str

m = ModelScopeModel(
    # 模型OSS路径，默认会被挂载到 /eas/workspace/model 路径下
    model_data=model_data_uri,
    # 使PAI提供的vllm推理镜像
    image_uri="eas-registry-vpc.{}.cr.aliyuncs.com/pai-eas/chat-llm-webui:3.0-vllm".format(
        sess.region_id
    ),
    # 推理服务执行前安装依赖
    requirements=["ms-swift>=1.6.0"],
    # 推理服务执行命令
    command="swift deploy --ckpt_dir /eas/workspace/model/ --port 8000 --host 0.0.0.0",
    port=8000,
)

predictor = m.deploy(
    # 推理服务名称
    service_name="qwen1_5_7b_chat_{}".format(random_str(8)),
    # 推理服务使用的机器实例
    instance_type="ecs.gn6e-c12g1.3xlarge",  # 1 * NVIDIA V100 (32 GB GPU Memory)
    # instance_type="ml.gu7i.c16m60.1-gu30",      # 1 * GU30 GPU (24 GB GPU Memory)
)

print(predictor.service_name)

View the service detail by accessing the console URI: 
https://pai.console.aliyun.com/?regionId=cn-hangzhou#/eas/serviceDetail/qwen_7b_chat_v5/detail


### 调用模型推理服务

通过`swift deploy`部署的大语言模型，支持通过OpenAI风格的HTTP API进行调用。开发者可以通过openai SDK（推荐）或是使用`Predictor`提供的`raw_predict`方法调用推理服务。

In [ ]:
# 安装openai SDK.
!pip install -q openai

In [31]:
import openai


# 获取推理服务的地址和访问密钥
endpoint = os.path.join(predictor.internet_endpoint, "v1")
access_token = predictor.access_token

client = openai.OpenAI(
    base_url=endpoint,
    api_key=access_token,
)

# 调用流式推理服务接口
completion = client.chat.completions.create(
    model="qwen1half-7b-chat",
    messages=[
        {"role": "user", "content": "一句话介绍一下你自己"},
    ],
    max_tokens=128,
    stream=True,
)

for chunk in completion:
    if not chunk.choices:
        continue
    content = chunk.choices[0].delta.content
    if content:
        print(content, end="")

我是一个人工智能助手，能够回答问题、提供建议、生成代码、聊天等任务，帮助用户解决问题和提高效率。

Answer: I am an AI assistant that can answer questions, provide suggestions, generate code, chat, and help users solve problems and improve efficiency.

用户也可以使用`deploy`方法返回的`predictor`对象，直接调用推理服务。

In [18]:
import json

resp = predictor.raw_predict(
    path="/v1/chat/completions",
    method="POST",
    data={
        "model": "qwen-7b-chat",
        "messages": [{"role": "user", "content": "一句话介绍一下你自己"}],
    },
)
print(json.dumps(resp.json(), indent=4, ensure_ascii=False))

{
    "model": "qwen-7b-chat",
    "choices": [
        {
            "index": 0,
            "message": {
                "role": "assistant",
                "content": "我是一个人工智能助手，可以回答问题、提供信息、进行对话等。我能够帮助用户解决问题，提供有用的信息和建议，以及进行各种任务。"
            },
            "finish_reason": "stop"
        }
    ],
    "usage": {
        "prompt_tokens": 22,
        "completion_tokens": 34,
        "total_tokens": 56
    },
    "id": "chatcmpl-516695f2072d4d3c9aa4bb7b5a5962f2",
    "object": "chat.completion",
    "created": 1707296083
}


## 总结

通过当前文档，我们了解了如何基于PAI Python SDK在PAI上使用ModelScope Swift框架进行模型微调训练和部署。通过PAI Python SDK，开发者可以轻松得使用使用各种开源框架完成模型的开发和部署，包括Swift，TensorFlow，PyTorch，HuggingFace transformers等，开发者可以通过[文档](https://alipai.readthedocs.io/)和[示例仓库](https://github.com/aliyun/pai-examples)了解更多关于PAI Python SDK的使用方法。



## 参考文档

- 阿里云机器学习平台PAI: https://www.aliyun.com/product/bigdata/learn

- ModelScope Swift文档：https://github.com/modelscope/swift/blob/main/docs/source/GetStarted/%E5%BF%AB%E9%80%9F%E4%BD%BF%E7%94%A8.md

- PAI Python SDK文档：https://alipai.readthedocs.io/

- PAI示例仓库：https://github.com/aliyun/pai-examples

